In [1]:
import pandas as pd
import skfuzzy as fuzz
import numpy as np
from skfuzzy import control as ctrl
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_pickle('choysum_data.pkl')
df.drop(columns = ['Windspeed'], inplace=True)
df = df[['Humidity', 'Moisture', 'Temperature', 'Light_intensity', 'Heat_index']]
df.head()

,Humidity,Moisture,Temperature,Light_intensity,Heat_index
Time_stamp,,,,,
2020-12-26 12:09:31,72.0,38.599998,26.000000,751.031993,27.368090
2020-12-26 12:10:23,73.0,37.900002,26.000000,1033.333252,25.590635
2020-12-26 12:11:16,72.0,37.900002,24.877772,1066.666626,25.590635
2020-12-26 12:11:52,72.0,38.299999,26.000000,933.333252,27.368090
2020-12-26 12:12:47,72.0,38.099998,26.000000,916.666626,27.368090


In [4]:
#Divide into train and test set

from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.3, random_state=1)

In [5]:
def get_range(data):
    data_range = pd.DataFrame(columns = data.columns, index = ['max', 'min'])
    data_range.loc['max'] = [data[col].max() for col in data.columns]
    data_range.loc['min'] = [data[col].min() for col in data.columns]
    return data_range

range_df = get_range(df)
range_df

,Humidity,Moisture,Temperature,Light_intensity,Heat_index
max,85,40,28,3283.33,30.6661
min,63,20,23,0,23.3398


In [8]:
#Specifying the numpy range of the universe of discourse
universe = []
for col in range_df.columns:
        universe_col = np.arange(np.floor(range_df[col].min()), np.ceil(range_df[col].max()+1), 0.1)
        universe.append(universe_col)

## Define antecedents and consequents

In [15]:
# using columns names to declare antecedents and consequents

Humidity = ctrl.Antecedent(universe[0], 'Humidity')
Moisture = ctrl.Antecedent(universe[1], 'Moisture')
Temperature = ctrl.Antecedent(universe[2], 'Temperature')
Light_intensity = ctrl.Antecedent(universe[3], 'Light_intensity')
Heat_index = ctrl.Consequent(universe[4], 'Heat_index')

In [16]:
def med(l, u):
    return np.median(np.arange(l, u))

In [13]:
universe[4][-1]

31.900000000000126

In [ ]:
med(63,85)

In [19]:
Humidity['low'] = fuzz.trimf(universe[0], [63, 63, med(63,86)])
Humidity['mid'] = fuzz.trimf(universe[0], [63, med(63,86), 86])
Humidity['high'] = fuzz.trimf(universe[0], [med(63,86), 86, 86])

Moisture['low'] = fuzz.trimf(universe[1], [20, 20, med(20,41)])
Moisture['mid'] = fuzz.trimf(universe[1], [20, med(20,41), 41])
Moisture['high'] = fuzz.trimf(universe[1], [med(20,41), 41, 41])

Temperature['low'] = fuzz.trimf(universe[2], [23, 23, med(23,29)])
Temperature['mid'] = fuzz.trimf(universe[2], [23, med(23,29), 29])
Temperature['high'] = fuzz.trimf(universe[2], [med(23,29), 29, 29])


Light_intensity['low'] = fuzz.trimf(universe[3], [0, 0, med(0,3285)])
Light_intensity['mid'] = fuzz.trimf(universe[3], [0, med(0,3285), 3285])
Light_intensity['high'] = fuzz.trimf(universe[3], [med(0,3285), 3285, 3285])

Heat_index['low'] = fuzz.trimf(universe[4], [23, 23, med(23,32)])
Heat_index['mid'] = fuzz.trimf(universe[4], [23, med(23,32), 32])
Heat_index['high'] = fuzz.trimf(universe[4], [med(23,32), 32, 32])

## Define membership functions

- degree of membership for X of feature 1(sepallength)

In [20]:
#renaming for interpolation

Hum_lo = fuzz.trimf(universe[0], [63, 63, med(63,86)])
Hum_mid = fuzz.trimf(universe[0], [63, med(63,86), 86])
Hum_hi = fuzz.trimf(universe[0], [med(63,86), 86, 86])

Moist_lo = fuzz.trimf(universe[1], [20, 20, med(20,41)])
Moist_mid= fuzz.trimf(universe[1], [20, med(20,41), 41])
Moist_hi = fuzz.trimf(universe[1], [med(20,41), 41, 41])

Temp_lo = fuzz.trimf(universe[2], [23, 23, med(23,29)])
Temp_mid = fuzz.trimf(universe[2], [23, med(23,29), 29])
Temp_hi = fuzz.trimf(universe[2], [med(23,29), 29, 29])


Light_lo = fuzz.trimf(universe[3], [0, 0, med(0,3285)])
Light_mid = fuzz.trimf(universe[3], [0, med(0,3285), 3285])
Light_hi = fuzz.trimf(universe[3], [med(0,3285), 3285, 3285])

Heat_lo= fuzz.trimf(universe[4], [23, 23, med(23,32)])
Heat_mid = fuzz.trimf(universe[4], [23, med(23,32), 32])
Heat_hi= fuzz.trimf(universe[4], [med(23,32), 32, 32])

In [43]:
hum_df = df_train.loc[:,['Humidity']]                  #create new dataframe and populate with columns

hum_df['low'] = fuzz.interp_membership(universe[0], Hum_lo, df_train.iloc[:,0])
hum_df['mid'] = fuzz.interp_membership(universe[0], Hum_mid, df_train.iloc[:,0])
hum_df['high'] = fuzz.interp_membership(universe[0], Hum_hi, df_train.iloc[:,0])

hum_df['membership'] = hum_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
hum_df['degree'] = hum_df.loc[:, ['low', 'mid', 'high']].max(axis = 1)

hum_df.head(2)

,Humidity,low,mid,high,membership,degree
Time_stamp,,,,,,
2021-01-01 07:14:40,75.0,0.000000,0.916667,0.083333,mid,0.916667
2021-01-02 12:39:21,64.0,0.909091,0.090909,0.000000,low,0.909091


In [42]:
hum_df.groupby(by='membership')['membership'].count()

membership
high      772
low       559
mid     15165
Name: membership, dtype: int64

- degree of membership for X of feature 2(sepalwidth)

In [46]:
moist_df = df_train.loc[:,['Moisture']]                  #create new dataframe and populate with columns

moist_df['low'] = fuzz.interp_membership(universe[1], Moist_lo, df_train.iloc[:,1])
moist_df['mid'] = fuzz.interp_membership(universe[1], Moist_mid, df_train.iloc[:,1])
moist_df['high'] = fuzz.interp_membership(universe[1], Moist_hi, df_train.iloc[:,1])

moist_df['membership'] = moist_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
moist_df['membership'] = moist_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
moist_df['degree'] = moist_df.loc[:, ['low', 'mid', 'high']].max(axis = 1)

moist_df.head(3)

,Moisture,low,mid,high,membership,degree
Time_stamp,,,,,,
2021-01-01 07:14:40,40.0,0.0,0.090909,0.909091,high,0.909091
2021-01-02 12:39:21,40.0,0.0,0.090909,0.909091,high,0.909091
2021-01-04 22:23:48,40.0,0.0,0.090909,0.909091,high,0.909091


In [45]:
moist_df.groupby(by='membership')['membership'].count()

membership
high    15709
low        23
mid       764
Name: membership, dtype: int64

In [49]:
#rules for temperature

temp_df = df_train.loc[:,['Temperature']]                  #create new dataframe and populate with columns

temp_df['low'] = fuzz.interp_membership(universe[2], Temp_lo, df_train.iloc[:,2])
temp_df['mid'] = fuzz.interp_membership(universe[2], Temp_mid, df_train.iloc[:,2])
temp_df['high'] = fuzz.interp_membership(universe[2], Temp_hi, df_train.iloc[:,2])

temp_df['membership'] = temp_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
temp_df['membership'] = temp_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
temp_df['degree'] = temp_df.loc[:, ['low', 'mid', 'high']].max(axis = 1)

temp_df.head(3)

,Temperature,low,mid,high,membership,degree
Time_stamp,,,,,,
2021-01-01 07:14:40,24.0,0.6,0.400000,0.000000,low,0.600000
2021-01-02 12:39:21,27.0,0.0,0.571429,0.428571,mid,0.571429
2021-01-04 22:23:48,25.0,0.2,0.800000,0.000000,mid,0.800000


In [50]:
temp_df.groupby(by='membership')['membership'].count()

membership
high       68
low      5594
mid     10834
Name: membership, dtype: int64

In [51]:
#rules for temperature

light_df = df_train.loc[:,['Temperature']]                  #create new dataframe and populate with columns

light_df['low'] = fuzz.interp_membership(universe[3], Light_lo, df_train.iloc[:,3])
light_df['mid'] = fuzz.interp_membership(universe[3], Light_mid, df_train.iloc[:,3])
light_df['high'] = fuzz.interp_membership(universe[3], Light_hi, df_train.iloc[:,3])

light_df['membership'] = light_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
light_df['membership'] = light_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
light_df['degree'] = light_df.loc[:, ['low', 'mid', 'high']].max(axis = 1)

light_df.head(3)

,Temperature,low,mid,high,membership,degree
Time_stamp,,,,,,
2021-01-01 07:14:40,24.0,0.903573,0.096427,0.0,low,0.903573
2021-01-02 12:39:21,27.0,0.081405,0.918595,0.0,mid,0.918595
2021-01-04 22:23:48,25.0,1.000000,0.000000,0.0,low,1.000000


In [52]:
light_df.groupby(by='membership')['membership'].count()

membership
high      407
low     11087
mid      5002
Name: membership, dtype: int64

In [53]:
#rules for temperature

heat_df = df_train.loc[:,['Heat_index']]                  #create new dataframe and populate with columns

heat_df['low'] = fuzz.interp_membership(universe[4], Heat_lo, df_train.iloc[:,4])
heat_df['mid'] = fuzz.interp_membership(universe[4], Heat_mid, df_train.iloc[:,4])
heat_df['high'] = fuzz.interp_membership(universe[4], Heat_hi, df_train.iloc[:,4])

heat_df['membership'] = heat_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
heat_df['membership'] = heat_df.loc[:, ['low', 'mid', 'high']].idxmax(axis = 1)
heat_df['degree'] = heat_df.loc[:, ['low', 'mid', 'high']].max(axis = 1)

heat_df.head(3)

,Heat_index,low,mid,high,membership,degree
Time_stamp,,,,,,
2021-01-01 07:14:40,24.413645,0.646589,0.353411,0.00000,low,0.646589
2021-01-02 12:39:21,28.377401,0.000000,0.724520,0.27548,mid,0.724520
2021-01-04 22:23:48,25.539745,0.365064,0.634936,0.00000,mid,0.634936


In [54]:
heat_df.groupby(by='membership')['membership'].count()

membership
high       74
low      5594
mid     10828
Name: membership, dtype: int64

## Build the Rules

In [55]:
rule_df = df_train.copy()
rule_df['Humidity'] = hum_df['membership']
rule_df['Moisture'] = moist_df['membership']
rule_df['Temperature'] = temp_df['membership']
rule_df['Light_intensity'] = light_df['membership']
rule_df['Heat_index'] = heat_df['membership']

rule_df['degree'] = hum_df['degree']*moist_df['degree']\
                    *temp_df['degree']*light_df['degree']\
                    *heat_df['degree']
rule_df.head()

,Humidity,Moisture,Temperature,Light_intensity,Heat_index,degree
Time_stamp,,,,,,
2021-01-01 07:14:40,mid,high,low,low,low,0.292120
2021-01-02 12:39:21,low,high,mid,mid,mid,0.314305
2021-01-04 22:23:48,mid,high,mid,low,mid,0.384810
2021-01-16 00:49:42,mid,high,low,low,low,0.167441
2021-01-16 14:35:38,mid,high,mid,mid,mid,0.284359


In [ ]:
#For comparison
#antecedent_unique = rule_df.loc[:, ['sepallength', 'sepalwidth','petallength', 'petalwidth']].drop_duplicates()

In [60]:
rule_fuzzy = rule_df.groupby(['Humidity', 'Moisture', 'Temperature', 'Light_intensity']).max()
rule_fuzzy = rule_fuzzy.reset_index()
rule_fuzzy

,Humidity,Moisture,Temperature,Light_intensity,Heat_index,degree
0,high,high,low,low,low,0.243150
1,high,high,low,mid,low,0.368672
2,high,high,mid,high,mid,0.229777
3,high,high,mid,low,mid,0.283214
4,high,high,mid,mid,mid,0.262204
5,high,mid,low,low,low,0.212598
6,high,mid,low,mid,low,0.208725
7,high,mid,mid,low,mid,0.286612
8,high,mid,mid,mid,mid,0.095424
9,low,high,high,mid,high,0.315527


In [61]:
rules = []
for idx, series in rule_fuzzy.iterrows():
    rule_i = ctrl.Rule(antecedent = (Humidity[series.values[0]] & Moisture[series.values[1]]           \
                                      & Temperature[series.values[2]] & Light_intensity[series.values[3]] ),        \
                         consequent = Heat_index[series.values[4]])
    rules.append(rule_i)

In [62]:
rules[0]

IF ((Humidity[high] AND Moisture[high]) AND Temperature[low]) AND Light_intensity[low] THEN Heat_index[low]
	AND aggregation function : fmin
	OR aggregation function  : fmax

In [ ]:
ctrSys = ctrl.ControlSystem(rules = rules)
simulator = ctrl.ControlSystemSimulation(ctrSys)

In [ ]:
simulator.input['sepal_length'] = 5.9
simulator.input['sepal_width'] = 3.2
simulator.input['petal_length'] = 4.8
simulator.input['petal_width'] = 1.8
simulator.compute()

In [ ]:
group.view(sim = simulator)
print("class", simulator.output['group'])